# Tracking MCMC Workflow
This notebook attempts to track the workflow of MCMC. This notebook has MANY purposes.

This notebook refers to the CURRENT TAG o1_lalinference_20160402

The following describes the process by which MCMC samples PE space and generates a precessing template.

First, [LALInferenceTemplateXLALSimInspiralChooseWaveform](http://software.ligo.org/docs/lalsuite/lalinference/group___l_a_l_inference_template__h.html#ga8725ff30be0c84ea6be1e1cfb3a317ea) is called. This function then calls 
[LALSimInspiralTransformPrecessingNewInitialConditions](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_spin_taylor__c.html#gacde647cce8c72a41d1dd0ac9e9b385da) in order to transform spin angle parameters into inclination, S1 and S2 so that these can be fed into 
[XLALSimInspiralChooseTDWaveformFromCache](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_waveform_cache__h.html#ga21112bbad5c769738c19ddb56701e6ae) or [XLALSimInspiralChooseFDWaveformFromCache](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_waveform_cache__h.html#ga1344d670f724f42a79e3044752fd1639) and by extension [XLALSimInspiralChooseTDWaveform](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral__c.html#gaf8e3028cce411328ddc1a3efa6f43cb2) or [XLALSimInspiralChooseFDWaveform](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral__c.html#gaf56ff1e81adcb147e481328f4e194f5a) in order to generate the template. For precessing templates, the following function is called in order to rotate the orientation of the system from OrbitalL  ( L=(0,0,1) N = (-sin(inc),0,cos(inc)) into an orientation where L is in the x-z plane and N along z (i.e. N=0,0,1)==>  [XLALSimInspiralInitialConditionsPrecessingApproxs](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_spin_taylor__c.html#gadc3d6ebbe9e62c8d7ea93b489db00bdb)
This output is *required* by precessing waveforms routines. This then gets passed into the waveform driver routine such as [XLALSimInspiralSpinTaylorT4](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_spin_taylor__c.html#ga43f8a11e72b41797fef44c7b2134a1ba)

The main point here is that N does NOT equal (-sin(inc),0,cos(inc)) at the end of all the rotations of LALSimInspiralTransformPrecessingNewInitialConditions despite the documentation claiming that it does. It in fact is  (sin(inc),0,cos(inc)). At no point is a rotation done to fix this (as far as I can see) before going into XLALSimInspiralInitialConditionsPrecessingApproxs which then does a rotation that WILL NOT get N to be along Z but in some arbitrary direction. Therefore, N and L will not oriented in such a way as is expect from the precessing waveform driving functions. 

As salvo discovered the issue appears to be rotation 6 done in LALSimInspiralTransformPrecessingNewInitialConditions
in the code below this rotation has is surrounded by two astericks. That rotation should NOT be -phiN but LAL_PI-phiN.

I realize that this links to doxygen that may be for the current master branch or out of date. Therefore, I am directly linking the code of the two functions in question for the cgit. In addition, I am fairly convinced of the above workflow for template generation by MCMC. If someone would like to correct me and say these functions are not called that is fine, or that the orientations are in fact fine and it is a documentation (major) flaw that is fine. Otherwise I am pretty sure that the precessing waveform driving routines are being passed S1 and S2 values calculated in a frame that they do NOT expect or want.

[o1_lalinference_20160402](https://versions.ligo.org/cgit/lalsuite/commit/lalsimulation/src/LALSimInspiralSpinTaylor.c?h=o1_lalinference_20160402)

```C\n
/**
 * Function to specify the desired orientation of a precessing binary in terms
 * of several angles and then compute the vector components with respect to
 * orbital angular momentum as needed to specify binary configuration for
 * ChooseTDWaveform.
 *
 * Input:
 * thetaJN is the inclination between total angular momentum (J) and the
 * direction of propagation (N)
 * theta1 and theta2 are the inclinations of S1 and S2
 * measured from the Newtonian orbital angular momentum (L_N)
 * phi12 is the difference in azimuthal angles of S1 and S2.
 * chi1, chi2 are the dimensionless spin magnitudes ( \f$0 \le chi1,2 \le 1\f$)
 * phiJL is the azimuthal angle of L_N on its cone about J.
 * m1, m2, f_ref are the component masses and reference GW frequency,
 * they are needed to compute the magnitude of L_N, and thus J.
 *
 * Output:
 * incl - inclination angle of L_N relative to N
 * x, y, z components S1 and S2 (unit spin vectors times their
 * dimensionless spin magnitudes - i.e. they have unit magnitude for
 * extremal BHs and smaller magnitude for slower spins).
 *
 * NOTE: Here the \"total\" angular momentum is computed as
 * J = L_N(1+l1PN) + S1 + S2
 * where L_N is the Newtonian orbital angular momentum and l1PN its
 * relative 1PN corrections. In fact, there are
 * PN corrections to L which contribute to J that are NOT ACCOUNTED FOR
 * in this function. This is done so to avoid complications with spin-orbit
 * contributions to L, which would require the full knowledge fo the orbital motion,
 * not just the evolution of L (see e.g.  eq.2.9c of arXiv:gr-qc/9506022).
 * Also, it is believed that the difference in Jhat
 * with or without these PN corrections to L is quite small.
 *
 * NOTE: fRef = 0 is not a valid choice. If you will pass fRef=0 into
 * ChooseWaveform, then here pass in f_min, the starting GW frequency
 *
 * Reviewed completed on git hash ...
 */
int XLALSimInspiralTransformPrecessingNewInitialConditions(
		REAL8 *incl,	/**< Inclination angle of L_N (returned) */
		REAL8 *S1x,	/**< S1 x component (returned) */
		REAL8 *S1y,	/**< S1 y component (returned) */
		REAL8 *S1z,	/**< S1 z component (returned) */
		REAL8 *S2x,	/**< S2 x component (returned) */
		REAL8 *S2y,	/**< S2 y component (returned) */
		REAL8 *S2z,	/**< S2 z component (returned) */
		const REAL8 thetaJN, 	/**< zenith angle between J and N (rad) */
		const REAL8 phiJL,  	/**< azimuthal angle of L_N on its cone about J (rad) */
		const REAL8 theta1,  	/**< zenith angle between S1 and LNhat (rad) */
		const REAL8 theta2,  	/**< zenith angle between S2 and LNhat (rad) */
		const REAL8 phi12,  	/**< difference in azimuthal angle btwn S1, S2 (rad) */
		const REAL8 chi1,	/**< dimensionless spin of body 1 */
		const REAL8 chi2,	/**< dimensionless spin of body 2 */
		const REAL8 m1_SI,	/**< mass of body 1 (kg) */
		const REAL8 m2_SI,	/**< mass of body 2 (kg) */
		const REAL8 fRef	/**< reference GW frequency (Hz) */
		)
{
	/* Check that fRef is sane */
	if( fRef == 0. )
	{
		XLALPrintError("XLAL Error - %s: fRef=0 is invalid. Please pass in the starting GW frequency instead.\n", __func__);
		XLAL_ERROR(XLAL_EINVAL);
	}

	REAL8 m1, m2, eta, v0, theta0, phi0, Jnorm, tmp1, tmp2;
	REAL8 Jhatx, Jhaty, Jhatz, LNhx, LNhy, LNhz, Jx, Jy, Jz, Lmag;
	REAL8 s1hatx,s1haty,s1hatz,s2hatx,s2haty,s2hatz;
	REAL8 s1x, s1y, s1z, s2x, s2y, s2z;

	/* Starting frame: LNhat is along the z-axis and the unit
	 * spin vectors are defined from the angles relative to LNhat.
	 * Note that we put s1hat in the x-z plane, and phi12
	 * sets the azimuthal angle of s2hat measured from the x-axis.
	 */
	LNhx = 0.;
	LNhy = 0.;
	LNhz = 1.;
	/* Spins are given wrt to L, but still we cannot fill the spin as we do not know
	 * what will be the relative orientation of L and N.
	 */
	s1hatx = sin(theta1);
	s1haty = 0.;
	s1hatz = cos(theta1);
	s2hatx = sin(theta2) * cos(phi12);
	s2haty = sin(theta2) * sin(phi12);
	s2hatz = cos(theta2);

	/* Define several internal variables needed for magnitudes */
	m1 = m1_SI/LAL_MSUN_SI;
	m2 = m2_SI/LAL_MSUN_SI;
	eta=m1*m2/(m1+m2)/(m1+m2);
	// v parameter at reference point
	v0 = cbrt( (m1+m2) * LAL_MTSUN_SI *LAL_PI * fRef );

	/* Define S1, S2, J with proper magnitudes */
	Lmag = XLALSimInspiralLN(m1+m2,eta,v0)*(1+v0*v0*XLALSimInspiralL_2PN(eta));
	s1x = m1 * m1 * chi1 * s1hatx;
	s1y = m1 * m1 * chi1 * s1haty;
	s1z = m1 * m1 * chi1 * s1hatz;
	s2x = m2 * m2 * chi2 * s2hatx;
	s2y = m2 * m2 * chi2 * s2haty;
	s2z = m2 * m2 * chi2 * s2hatz;
	Jx = s1x + s2x;
	Jy = s1y + s2y;
	Jz = Lmag * LNhz + s1z + s2z;

	/* Normalize J to Jhat, find it's angles in starting frame */
	Jnorm = sqrt( Jx*Jx + Jy*Jy + Jz*Jz);
	Jhatx = Jx / Jnorm;
	Jhaty = Jy / Jnorm;
	Jhatz = Jz / Jnorm;
	theta0 = acos(Jhatz);
	phi0 = atan2(Jhaty, Jhatx);

	/* Rotation 1: Rotate about z-axis by -phi0 to put Jhat in x-z plane */
	ROTATEZ(-phi0, LNhx, LNhy, LNhz);
	ROTATEZ(-phi0, s1hatx, s1haty, s1hatz);
	ROTATEZ(-phi0, s2hatx, s2haty, s2hatz);

	/* Rotation 2: Rotate about new y-axis by -theta0
	 * to put Jhat along z-axis
	 */
	ROTATEY(-theta0, LNhx, LNhy, LNhz);
	ROTATEY(-theta0, s1hatx, s1haty, s1hatz);
	ROTATEY(-theta0, s2hatx, s2haty, s2hatz);

	/* Rotation 3: Rotate about new z-axis by phiJL to put L at desired
	 * azimuth about J. Note that is currently in x-z plane towards -x
	 * (i.e. azimuth=pi). Hence we rotate about z by phiJL - LAL_PI
	 */
	ROTATEZ(phiJL - LAL_PI, LNhx, LNhy, LNhz);
	ROTATEZ(phiJL - LAL_PI, s1hatx, s1haty, s1hatz);
	ROTATEZ(phiJL - LAL_PI, s2hatx, s2haty, s2hatz);

	/* The cosinus of the angle between L and N is the scalar
         * product of the two vectors. We do need to perform additional
         * rotation to compute it
         */
	*incl=acos(-sin(thetaJN)*LNhx+cos(thetaJN)*LNhz); //output

	/* Rotation 4-5: Now J is along z and N in x-z plane, inclined from J
         * by thetaJN. Now we bring L into the z axis to get
         * spin components.
	 */
	REAL8 Nx=-sin(thetaJN);
	REAL8 Ny=0.;
	REAL8 Nz=cos(thetaJN);
	REAL8 thetaLJ = acos(LNhz);
	REAL8 phiL    = atan2(LNhy, LNhx);

	ROTATEZ(-phiL, Nx, Ny, Nz);
	ROTATEZ(-phiL, s1hatx, s1haty, s1hatz);
	ROTATEZ(-phiL, s2hatx, s2haty, s2hatz);
	ROTATEY(-thetaLJ, Nx, Ny, Nz);
	ROTATEY(-thetaLJ, s1hatx, s1haty, s1hatz);
	ROTATEY(-thetaLJ, s2hatx, s2haty, s2hatz);

	/* Rotation 6: Now L is along z and we have to bring N
	 * in the x-z plane.
	 */
	REAL8 phiN = atan2(Ny, Nx);
	**ROTATEZ(-phiN, s1hatx, s1haty, s1hatz)**;
	**ROTATEZ(-phiN, s2hatx, s2haty, s2hatz)**;

	/* Set pointers to rotated spin vectors */
	*S1x = s1hatx*chi1;
	*S1y = s1haty*chi1;
	*S1z = s1hatz*chi1;
	*S2x = s2hatx*chi2;
	*S2y = s2haty*chi2;
	*S2z = s2hatz*chi2;

	//Uncomment the following lines for a check of the rotation
	/*printf("LNhat should be along z, N in the x-z plane\n");
	ROTATEZ(-phiL,    LNhx, LNhy, LNhz);
	ROTATEY(-thetaLJ, LNhx, LNhy, LNhz);
	ROTATEZ(-phiN,    LNhx, LNhy, LNhz);
	ROTATEZ(-phiN,    Nx, Ny, LNz);
	ROTATEZ(-phi0, Jhatx, Jhaty, Jhatz);
	ROTATEY(-theta0, Jhatx, Jhaty, Jhatz);
	ROTATEZ(phiJL - LAL_PI, Jhatx, Jhaty, Jhatz);
	ROTATEZ(-phiL, Jhatx, Jhaty, Jhatz);
	ROTATEY(-thetaLJ, Jhatx, Jhaty, Jhatz);
	printf("LNhat: %12.4e  %12.4e  %12.4e\n",LNhx,LNhy,LNhz);
	printf("       %12.4e  %12.4e  %12.4e\n",0.,0.,1.);
	printf("N:     %12.4e  %12.4e  %12.4e\n",Nx,Ny,Nz);
	printf("       %12.4e  %12.4e  %12.4e\n",-sin(*incl),0.,cos(*incl));
	printf("J.Lhat  i: %12.4e f: %12.4e\n",Jz,Jnorm*Jhatz);
	printf("S1.L    i: %12.4e f: %12.4e\n",chi1*cos(theta1),*S1z);
	printf("S2.L    i: %12.4e f: %12.4e\n",chi2*cos(theta2),*S2z);
	printf("S1.S2   i: %12.4e f: %12.4e\n",chi1*chi2*(sin(theta1)*sin(theta2)*cos(phi12)+cos(theta1)*cos(theta2)),(*S1x)*(*S2x)+(*S1y)*(*S2y)+(*S1z)*(*S2z));*/

	return XLAL_SUCCESS;
}


/**
 * Function to specify the desired orientation of the spin components of
 * a precessing binary.
 *
 * Input:
 * x, y, z components S1 and S2 wrt
 * * total J for axisChoice        = LAL_SIM_INSPIRAL_FRAME_AXIS_TOTAL_J
 * * reference L for axisChoice    = LAL_SIM_INSPIRAL_FRAME_AXIS_ORBITAL_L (default)
 * * view direction for axisChoice = LAL_SIM_INSPIRAL_FRAME_AXIS_VIEW
 * incl is the angle between
 * * J and N (Jx \f$\propto sin(inc)\f$, Jy=0) for axisChoice = LAL_SIM_INSPIRAL_FRAME_AXIS_TOTAL_J
 * * L and N (Nx \f$\propto sin(inc)\f$, Ly=0) for axisChoice = LAL_SIM_INSPIRAL_FRAME_AXIS_ORBITAL_L (default)
 * * L and N (Lx \f$\propto sin(inc)\f$, Ly=0) for axisChoice = LAL_SIM_INSPIRAL_FRAME_AXIS_VIEW
 * m1, m2, f_ref are the component masses and reference GW frequency,
 * they are needed to compute the magnitude of L_N
 *
 * Output:
 * x, y, z components S1 and S2 wrt N
 * inc angle between L and N
 * as required by precessing waveforms routines.
 *
 * NOTE: Here the \"total\" angular momentum is computed as
 * J = L_N(1+L_1PN) + S1 + S2
 * where L_N(1+L_1PN) is the 1PN-corrected orbital angular momentum,
 * which is parallel to Newtonian angular momentum.
 * PN corrections to L from 1.5PN order on (wrt to Newtonian value)
 * are NOT ACCOUNTED FOR in this function.
 */
int XLALSimInspiralInitialConditionsPrecessingApproxs(
		REAL8 *inc,	/**< inclination angle (returned) */
		REAL8 *S1x,	/**< S1 x component (returned) */
		REAL8 *S1y,	/**< S1 y component (returned) */
		REAL8 *S1z,	/**< S1 z component (returned) */
		REAL8 *S2x,	/**< S2 x component (returned) */
		REAL8 *S2y,	/**< S2 y component (returned) */
		REAL8 *S2z,	/**< S2 z component (returned) */
		const REAL8 inclIn, /**< Inclination angle in input */
		const REAL8 S1xIn,  /**< S1 x component */
		const REAL8 S1yIn,  /**< S1 y component */
		const REAL8 S1zIn,  /**< S1 z component */
		const REAL8 S2xIn,  /**< S2 x component */
		const REAL8 S2yIn,  /**< S2 y component */
		const REAL8 S2zIn,  /**< S2 z component */
		const REAL8 m1,	    /**< mass of body 1 (kg) */
		const REAL8 m2,	    /**< mass of body 2 (kg) */
		const REAL8 fRef,   /**< reference GW frequency (Hz) */
		LALSimInspiralFrameAxis axisChoice  /**< Flag to identify axis wrt which spin components are given. Pass in NULL (or None in python) for default (OrbitalL) */)
{
  REAL8 Lmag=0.;
  REAL8 Lx,Ly,Lxy2,Lz;
  REAL8 LNhx,LNhy,LNhz,phiLN;
  REAL8 v0,mass1,mass2,eta;
  REAL8 tmp1,tmp2;
  *S1x=S1xIn;
  *S1y=S1yIn;
  *S1z=S1zIn;
  *S2x=S2xIn;
  *S2y=S2yIn;
  *S2z=S2zIn;
  switch (axisChoice) {
  /* FRAME_AXIS_TOTAL_J
   * (spins wrt to J, inclIn is the angle between J and N: if
   * J=(0,0,1) N=(-sin(inclIn),0,cos(inclIn)))
   */
  case LAL_SIM_INSPIRAL_FRAME_AXIS_TOTAL_J:
    mass1=m1*LAL_MSUN_SI;
    mass2=m2*LAL_MSUN_SI;
    eta=mass1*mass2/(mass1+mass2)/(mass1+mass2);
    v0=cbrt(LAL_PI*fRef*(m1+m2)*LAL_MTSUN_SI);
    Lmag = XLALSimInspiralLN(mass1+mass2,eta,v0)*(1.+v0*v0*XLALSimInspiralL_2PN(eta));
    Lx  = -S1xIn*mass1*mass1-S2xIn*mass2*mass2;
    Ly  = -S1yIn*mass2*mass2-S2yIn*mass2*mass2;
    Lxy2= Lx*Lx+Ly*Ly;
    Lz=0.;
    if (Lmag*Lmag>=Lxy2) {
      Lz=sqrt(Lmag*Lmag-Lxy2);
      if ( Lz<(S1zIn*mass1*mass1+S2zIn*mass2*mass2) ) {
	XLALPrintError("** XLALSimInspiralInitialConditionsPrecessingApproxs ERROR *** for s1 (%12.4e  %12.4e  %12.4e)\n",S1xIn,S1yIn,S1zIn);
	XLALPrintError("                                                                   s2 (%12.4e  %12.4e  %12.4e)\n",S2xIn,S2yIn,S2zIn);
	XLALPrintError(" wrt to J for m: (%12.4e  %12.4e) and v= %12.4e\n",mass1,mass2,v0);
	XLALPrintError(" it is impossible to determine the sign of LNhz\n");
	XLAL_ERROR(XLAL_EDOM);
      }
    }
    else {
      XLALPrintError("** XLALSimInspiralInitialConditionsPrecessingApproxs ERROR *** unphysical values of s1 (%12.4e  %12.4e  %12.4e)\n",S1xIn,S1yIn,S1zIn);
      XLALPrintError("                                                                                    s2 (%12.4e  %12.4e  %12.4e)\n",S2xIn,S2yIn,S2zIn);
      XLALPrintError(" wrt to J for m: (%12.4e  %12.4e) and v= %12.4e\n",mass1,mass2,v0);
      XLAL_ERROR(XLAL_EDOM);
    }
    // 1PN corrections to L are collinear with LN
    // Rotation1: send N to z
    LNhx=Lx/Lmag;
    LNhy=Ly/Lmag;
    LNhz=Lz/Lmag;
    ROTATEY(inclIn,*S1x,*S1y,*S1z);
    ROTATEY(inclIn,*S2x,*S2y,*S2z);
    ROTATEY(inclIn,LNhx,LNhy,LNhz);
    *inc=acos(LNhz);
    //Rotation 2: send L into the x-z plane.
    phiLN=atan2(LNhy,LNhx);
    ROTATEZ(-phiLN,*S1x,*S1y,*S1z);
    ROTATEZ(-phiLN,*S2x,*S2y,*S2z);
    //It follows that in the frame in which N=(0,0,1), LNhat=(sin(inc),0,cos(inc))
    //For a check uncomment the following lines
    /*printf(" Check of LAL_SIM_INSPIRAL_FRAME_AXIS_TOTAL_J\n");
    REAL8 Nx=-sin(inclIn);
    REAL8 Ny=0.;
    REAL8 Nz=cos(inclIn);
    ROTATEY(inclIn,Nx,Ny,Nz);
    ROTATEZ(-phiLN,Nx,Ny,Nz);
    ROTATEZ(-phiLN,LNhx,LNhy,LNhz);
    printf("N:  %12.4e  %12.4e  %12.4e  norm: %12.4e\n",Nx,Ny,Nz,sqrt(Nx*Nx+Ny*Ny+Nz*Nz));
    printf("     %12.4e  %12.4e  %12.4e\n",0.,0.,1.);
    printf("LNh: %12.4e  %12.4e  %12.4e  norm: %12.4e\n",LNhx,LNhy,LNhz,sqrt(LNhx*LNhx+LNhy*LNhy+LNhz*LNhz));
    printf("     %12.4e  %12.4e  %12.4e\n",sin(*inc),0.,cos(*inc));
    printf("S1.L     %12.4e  %12.4e\n",(*S1x)*LNhx+(*S1y)*LNhy+(*S1z)*LNhz,S1zIn);
    printf("S2.L     %12.4e  %12.4e\n",(*S2x)*LNhx+(*S2y)*LNhy+(*S2z)*LNhz,S2zIn);
    printf("S1.S2    %12.4e  %12.4e\n",(*S1x)*(*S2x)+(*S1y)*(*S2y)+(*S1z)*(*S2z),S1xIn*S2xIn+S1yIn*S2yIn+S1zIn*S2zIn);*/
    break;

  /* FRAME_AXIS_VIEW (OLD default)
   * (spins wrt to view direction, inclIn is the angle between L and N: if
   * N=(0,0,1) LNhat=(sin(inclIn),0,cos(inclIn)) )
   */
  case LAL_SIM_INSPIRAL_FRAME_AXIS_VIEW:
    *inc=inclIn;
    break;
  /* FRAME_AXIS_ORBITAL_L (default)
   * (spins wrt to L, inclIn is the angle between L and N: if
   * LNhat=(0,0,1) N=(-sin(inclIn),0,cos(inclIn)) )
   */
  case LAL_SIM_INSPIRAL_FRAME_AXIS_ORBITAL_L:
  default:
    ROTATEY(inclIn,*S1x,*S1y,*S1z);
    ROTATEY(inclIn,*S2x,*S2y,*S2z);
    *inc=inclIn;
    //For a check uncomment the following lines
    /*printf(" Check of LAL_SIM_INSPIRAL_FRAME_AXIS_ORBITAL_L\n");
    REAL8 Nnx=-sin(inclIn);
    REAL8 Nny=0.;
    REAL8 Nnz=cos(inclIn);
    LNhx=0.;
    LNhy=0.;
    LNhz=1.;
    ROTATEY(inclIn,Nnx,Nny,Nnz);
    ROTATEY(inclIn,LNhx,LNhy,LNhz);
    printf("N:   %12.4e  %12.4e  %12.4e  norm: %12.4e\n",Nnx,Nny,Nnz,sqrt(Nnx*Nnx+Nny*Nny+Nnz*Nnz));
    printf("     %12.4e  %12.4e  %12.4e\n",0.,0.,1.);
    printf("LNh: %12.4e  %12.4e  %12.4e  norm: %12.4e\n",LNhx,LNhy,LNhz,sqrt(LNhx*LNhx+LNhy*LNhy+LNhz*LNhz));
    printf("     %12.4e  %12.4e  %12.4e\n",sin(*inc),0.,cos(*inc));
    printf("S1.L     %12.4e  %12.4e\n",(*S1x)*LNhx+(*S1y)*LNhy+(*S1z)*LNhz,S1zIn);
    printf("S2.L     %12.4e  %12.4e\n",(*S2x)*LNhx+(*S2y)*LNhy+(*S2z)*LNhz,S2zIn);
    printf("S1.S2    %12.4e  %12.4e\n",(*S1x)*(*S2x)+(*S1y)*(*S2y)+(*S1z)*(*S2z),S1xIn*S2xIn+S1yIn*S2yIn+S1zIn*S2zIn);*/
    break;
  }

  return XLAL_SUCCESS;
}
```
 
 
 First, we inject a signal. For the injection, MCMC uses the following workflow.

MCMC calls ===> LALInferenceInjectInspiralSignal which calls ===>
[XLALSimInspiralChooseTDWaveform](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral__c.html#gaf8e3028cce411328ddc1a3efa6f43cb2) ==>
does a giant switch..case.. over all the possible approximants. For a precessing waveform such as SpinTaylorT4 the inclination S1 and S2 from the injection XML gets passed through
[XLALSimInspiralInitialConditionsPrecessingApproxs](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_spin_taylor__c.html#gadc3d6ebbe9e62c8d7ea93b489db00bdb) which expects by default Input: x, y, z components S1 and S2 wrt reference L for axisChoice = LAL_SIM_INSPIRAL_FRAME_AXIS_ORBITAL_L/L and N (Nx ∝sin(inc), Ly=0) for axisChoice = LAL_SIM_INSPIRAL_FRAME_AXIS_ORBITAL_L (default) and outputs S1 and S2 Output: x, y, z components S1 and S2 wrt N and inc angle between L and N as required by precessing waveforms routines. This then gets passed into the waveform driver routine
[XLALSimInspiralSpinTaylorT4](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_spin_taylor__c.html#ga43f8a11e72b41797fef44c7b2134a1ba)